In [ ]:
import pypyodbc as odbc
import pandas as pd
import requests
import time

# SQL Server Connection
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

# Create connection object
conn = odbc.connect(connection_string)

# Get coins from database - pass connection object instead of string
coins_df = pd.read_sql(
    "SELECT DISTINCT id, name FROM bronze.coin_market WHERE id IS NOT NULL", 
    conn
)

# Fetch OHLC data for all coins
all_data = []
for index, row in coins_df.iterrows():
    coin_id = row['id']
    print(f"Fetching data for {coin_id}...")
    
    try:
            response = requests.get(f'https://api.coingecko.com/api/v3/coins/{coin_id}/ohlc', params={'vs_currency': 'usd', 'days': '365'})
            response.raise_for_status()
            
            coin_df = pd.DataFrame(response.json(), columns=['timestamp', 'open', 'high', 'low', 'close'])
            coin_df['coin_id'] = coin_id
            coin_df['coin_name'] = row.get('name', coin_id)
            coin_df['datetime'] = pd.to_datetime(coin_df['timestamp'], unit='ms')
            all_data.append(coin_df)
            
            print(f"✓ Successfully fetched {len(coin_df)} records for {coin_id}")
            time.sleep(20)
            
    except Exception as e:
            print(f"✗ Error fetching {coin_id}: {e}")
            continue

# Close connection
conn.close()

# Combine all data
ohlc_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Display results by coin with separators
if not ohlc_df.empty:
    for coin_id in ohlc_df['coin_id'].unique():
        coin_data = ohlc_df[ohlc_df['coin_id'] == coin_id]
        print(f"\n{'='*80}\nCOIN: {coin_id.upper()} - {coin_data['coin_name'].iloc[0]}\n{'='*80}")
        print(coin_data[['datetime', 'open', 'high', 'low', 'close']].head())
        print(f"Total records: {len(coin_data)}")
        
else:
    print("No data fetched.")

C:\Users\eBay\AppData\Local\Temp\ipykernel_23260\1315061028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coins_df = pd.read_sql(


Fetching data for aave...
✓ Successfully fetched 92 records for aave
Fetching data for avalanche-2...
✓ Successfully fetched 92 records for avalanche-2
Fetching data for binance-bridged-usdt-bnb-smart-chain...
✓ Successfully fetched 92 records for binance-bridged-usdt-bnb-smart-chain
Fetching data for binancecoin...
✓ Successfully fetched 92 records for binancecoin
Fetching data for bitcoin...
✓ Successfully fetched 92 records for bitcoin
Fetching data for bitcoin-cash...
✓ Successfully fetched 92 records for bitcoin-cash
Fetching data for bittensor...
✓ Successfully fetched 92 records for bittensor
Fetching data for canton-network...
✓ Successfully fetched 8 records for canton-network
Fetching data for cardano...
✓ Successfully fetched 92 records for cardano
Fetching data for chainlink...
✓ Successfully fetched 92 records for chainlink
Fetching data for coinbase-wrapped-btc...
✓ Successfully fetched 92 records for coinbase-wrapped-btc
Fetching data for crypto-com-chain...
✓ Successful

In [75]:
table_name = "bronze.candle_historical_data"
# SQL Server Connection
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

# Create connection object
connection = odbc.connect(connection_string)
sql = (f"""
SELECT coin_id, Max([datetime]) AS last_data_load, COUNT(*) AS number_of_data
FROM {table_name}
GROUP BY coin_id
"""
)
cursor = connection.cursor()


cursor.execute(sql)


all_coin_rows = cursor.fetchall()

all_coin = pd.DataFrame(all_coin_rows, columns=['coin_id','last_data_load', 'number_of_data'])
all_coin

,coin_id,last_data_load,number_of_data


In [76]:
filtered_coins_less92_datapoint = all_coin[
    (all_coin['coin_id'].isin(ohlc_df['coin_id'])) &
    (all_coin['number_of_data'] < 92)
]
filtered_coins_less92_datapoint

,coin_id,last_data_load,number_of_data


In [ ]:
ohlc_less_92_incremental = (
    ohlc_df
    .merge(filtered_coins_less92_datapoint, on='coin_id', how='left')
)
ohlc_less_92_incremental

,timestamp,open,high,low,close,coin_id,coin_name,datetime,last_data_load,number_of_data
0,1733875200000,259.54,286.89,250.83,273.28,aave,Aave,2024-12-11,NaN,NaN
1,1734220800000,274.81,391.90,260.97,366.00,aave,Aave,2024-12-15,NaN,NaN
2,1734566400000,365.57,397.54,340.71,340.71,aave,Aave,2024-12-19,NaN,NaN
3,1734912000000,340.01,348.72,272.76,316.81,aave,Aave,2024-12-23,NaN,NaN
4,1735257600000,317.60,387.27,310.13,337.46,aave,Aave,2024-12-27,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1778,1763942400000,9.37,9.52,9.04,9.44,leo-token,LEO Token,2025-11-24,NaN,NaN
1779,1764288000000,9.49,9.81,9.43,9.78,leo-token,LEO Token,2025-11-28,NaN,NaN
1780,1764633600000,9.79,9.91,9.68,9.85,leo-token,LEO Token,2025-12-02,NaN,NaN
1781,1764979200000,9.87,9.87,9.11,9.47,leo-token,LEO Token,2025-12-06,NaN,NaN


In [67]:
less_92datapoint_coins


[]

In [78]:
ohlc_new

,timestamp,open,high,low,close,coin_id,coin_name,datetime
0,1733875200000,259.54,286.89,250.83,273.28,aave,Aave,2024-12-11
1,1734220800000,274.81,391.90,260.97,366.00,aave,Aave,2024-12-15
2,1734566400000,365.57,397.54,340.71,340.71,aave,Aave,2024-12-19
3,1734912000000,340.01,348.72,272.76,316.81,aave,Aave,2024-12-23
4,1735257600000,317.60,387.27,310.13,337.46,aave,Aave,2024-12-27
...,...,...,...,...,...,...,...,...
4281,1763942400000,672.61,713.31,478.39,572.94,zcash,Zcash,2025-11-24
4282,1764288000000,572.46,590.43,485.75,489.07,zcash,Zcash,2025-11-28
4283,1764633600000,489.27,489.27,326.75,345.49,zcash,Zcash,2025-12-02
4284,1764979200000,344.93,403.75,303.31,359.53,zcash,Zcash,2025-12-06


In [80]:
historical_data_less_92_incremental = ohlc_df[
        ohlc_df['coin_id'].isin(filtered_coins_less92_datapoint['coin_id'])]

historical_data_less_92_incremental

,timestamp,open,high,low,close,coin_id,coin_name,datetime


In [81]:
ohlc_less_92_incremental = (
    ohlc_df
    .merge(filtered_coins_less92_datapoint, on='coin_id', how='left')
)

ohlc_less_92_incremental

,timestamp,open,high,low,close,coin_id,coin_name,datetime,last_data_load,number_of_data
0,1733875200000,259.54,286.89,250.83,273.28,aave,Aave,2024-12-11,NaN,NaN
1,1734220800000,274.81,391.90,260.97,366.00,aave,Aave,2024-12-15,NaN,NaN
2,1734566400000,365.57,397.54,340.71,340.71,aave,Aave,2024-12-19,NaN,NaN
3,1734912000000,340.01,348.72,272.76,316.81,aave,Aave,2024-12-23,NaN,NaN
4,1735257600000,317.60,387.27,310.13,337.46,aave,Aave,2024-12-27,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4281,1763942400000,672.61,713.31,478.39,572.94,zcash,Zcash,2025-11-24,NaN,NaN
4282,1764288000000,572.46,590.43,485.75,489.07,zcash,Zcash,2025-11-28,NaN,NaN
4283,1764633600000,489.27,489.27,326.75,345.49,zcash,Zcash,2025-12-02,NaN,NaN
4284,1764979200000,344.93,403.75,303.31,359.53,zcash,Zcash,2025-12-06,NaN,NaN


In [77]:
ohlc_coins = set(ohlc_df['coin_id'])
db_coins = set(all_coin['coin_id'])

new_coins = ohlc_coins - db_coins

less_92datapoint_coins = all_coin[all_coin['number_of_data'] < 92]['coin_id'].tolist()

full_data_coin = all_coin[all_coin['number_of_data'] >= 92]['coin_id'].tolist()

ohlc_new = ohlc_df[ohlc_df['coin_id'].isin(new_coins)]


if not ohlc_new.empty:
    params_list = [
        (
            row['timestamp'], row['open'], row['high'], row['low'], row['close'],
            row['coin_id'], row['coin_name'], row['datetime']
        )
        for _, row in ohlc_new.iterrows()
    ]

    load_query = f'''
    INSERT INTO {table_name} (
        [timestamp], [open], [high], [low], [close],
        coin_id, coin_name, [datetime]
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.executemany(load_query, params_list)
    connection.commit()
    print(f"🆕 {len(params_list)} rows inserted for new coins")


if  less_92datapoint_coins:

    filtered_coins_less92_datapoint = all_coin[
    (all_coin['coin_id'].isin(ohlc_df['coin_id'])) &
    (all_coin['number_of_data'] < 92)
]
    
    ohlc_less_92_incremental = (
    ohlc_df
    .merge(filtered_coins_less92_datapoint, on='coin_id', how='left')
)
    
    params_list = [
        (
            row['timestamp'], row['open'], row['high'], row['low'], row['close'],
            row['coin_id'], row['coin_name'], row['datetime']
        )
        for _, row in ohlc_less_92_incremental.iterrows()
    ]
    cursor.executemany(load_query, params_list)
    connection.commit()
    print(f"🔄 {len(params_list)} incremental rows inserted for partial coins")
else:
    print("ℹ️ No incremental rows to insert for partial coins")



ohlc_mature = ohlc_df[ohlc_df['coin_id'].isin(full_data_coin)]
if not ohlc_mature.empty:

    delete_earliest_92_datapoint_sql = f"""
    WITH data_occurrence AS (
    SELECT coin_id, COUNT(*) AS number_of_data
    FROM {table_name}
    GROUP BY coin_id
    HAVING COUNT(*) >= 92
),
Ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY coin_id ORDER BY [datetime] DESC) AS rn
    FROM {table_name}
)
DELETE Ranked
FROM Ranked
JOIN data_occurrence
  ON Ranked.coin_id = data_occurrence.coin_id
WHERE rn = 1
    """
    cursor.execute(delete_earliest_92_datapoint_sql)
    rows = cursor.fetchall()
    selected = len(rows)
    connection.commit()
    print(f"✅ {cursor.rowcount} oldest rows deleted for full data coins")

    
    # Keep only latest row per coin
    ohlc_latest = (
        ohlc_mature
        .sort_values('datetime')
        .groupby('coin_id', as_index=False)
        .tail(1)
    )
    params_list = [
        (
            row['timestamp'], row['open'], row['high'], row['low'], row['close'],
            row['coin_id'], row['coin_name'], row['datetime']
        )
        for _, row in ohlc_latest.iterrows()
    ]
    cursor.executemany(load_query, params_list)
    connection.commit()
    print(f"♻️ {len(params_list)} latest rows inserted for mature coins")

# -----------------------------
# CLOSE CONNECTION
# -----------------------------
cursor.close()
connection.close()
print("All done ✅")

🆕 4286 rows inserted for new coins
ℹ️ No incremental rows to insert for partial coins
All done ✅


In [ ]:
ohlc_coins = set(ohlc_df['coin_id'])
db_coins = set(all_coin['coin_id'])


filtered_coins_92_datapoint = all_coin[
    (all_coin['coin_id'].isin(ohlc_df['coin_id'])) &
    (all_coin['number_of_data'] >= 92)
]

ohlc_latest_92 = (
    filtered_coins_92_datapoint
    .sort_values('datetime')
    .groupby('coin_id', as_index=False)
    .tail(1)
)

filtered_coins_less92_datapoint = all_coin[
    (all_coin['coin_id'].isin(ohlc_df['coin_id'])) &
    (all_coin['number_of_data'] < 92)
]


ohlc_latest_92 = (
    filtered_coins_less92_datapoint
    .sort_values('datetime')
    .groupby('coin_id', as_index=False)
    .tail(1)
)

if not filtered_coins_92_datapoint.empty:
    delete_earliest_92_datapoint_sql = """
    WITH data_occurrence AS (
        SELECT coin_id, COUNT(*) AS number_of_data
        FROM bronze.candle_historical_data
        GROUP BY coin_id
        HAVING COUNT(*) >= 92
    ),
    Ranked AS (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY coin_id ORDER BY [datetime] DESC) AS rn
        FROM bronze.candle_historical_data
    )
    SELECT Ranked.*
    FROM Ranked
    JOIN data_occurrence
      ON Ranked.coin_id = data_occurrence.coin_id
    WHERE rn = 1
    """
    cursor.execute(delete_earliest_92_datapoint_sql)
    rows = cursor.fetchall()
    selected = len(rows)
    connection.commit()
    print(f"✅ {selected} latest rows have been selected.")


    

    params_list = [
        (
            row['timestamp'], row['open'], row['high'], row['low'], row['close'],
            row['coin_id'], row['coin_name'], row['datetime']
        )
        for _, row in ohlc_latest_92.iterrows()
    ]

    cursor.executemany(load_query, params_list)
    connection.commit()

    cursor.close()
    connection.close()

    print(f"{len(params_list)} new rows inserted into {table_name} ✅")

    
elif 
    


SyntaxError: invalid syntax (3532761917.py, line 71)

In [30]:
filtered_coins_92_datapoint

,coin_id,number_of_data
0,bitcoin-cash,92
1,ethereum,92
2,bitcoin,92
3,binance-bridged-usdt-bnb-smart-chain,92
4,chainlink,92
5,crypto-com-chain,92
6,ethena-staked-usde,92
7,binancecoin,92
8,dogecoin,92
9,coinbase-wrapped-btc,92
